# Colour viewer for HOYS lightcurves

* ***Shift + Enter on a code cell to run it and advance to the next cell.***

* ***If changing sliders, rerun the code block below the sliders***

* ***Plot windows are interactive, can use mouse to select and zoom into ranges***

In [ ]:
#load required modules
import os
import pandas as pd
import astropy
import numpy
import math
from astropy.time import Time
from astropy.timeseries import LombScargle
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = "presentation"
from plotly.offline import iplot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output

### List available lightcurve data and select from dropdown list
* Upload your .csv file to '/light_curve_csv_files'

In [ ]:
#read HOYS csv file
lc_folder = "./light_curve_csv_files/"
star_list=os.listdir(lc_folder)

lc_select = widgets.Dropdown(
    options=sorted(star_list),
    description='Select a light curve file:',
    style = {'description_width': 'initial'}
)
display(lc_select)

### Display head and tail of csv file containing all of the data

In [ ]:
print('selected datafile:',lc_select.value)
lc_data = pd.read_csv(os.path.join(lc_folder,lc_select.value),comment='#',delimiter=' ')
pd.set_option('display.max_columns', None)
#view file head and tail
lc_data

### Select minimum calibration error and median window for colours

In [ ]:
cal_error_sel=widgets.FloatSlider(
    value=0.1,
    min=0.05,
    max=0.2,
    step=0.01,
    description='Calibrated error [mag]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)
display(cal_error_sel)

med_window_sel=widgets.FloatSlider(
    value=1,
    min=0.5,
    max=10,
    step=0.1,
    description='Median filter window [days]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)
display(med_window_sel)

### Query lightcurve data and define median colours based on selections

In [ ]:
band=['B','V','R','I']
cal_error=cal_error_sel.value
med_window=med_window_sel.value

filtered_data=lc_data[(lc_data['filter'].isin(band)) & (lc_data['flags'] <= 4) & (lc_data['calibrated_error'] < cal_error) & (lc_data['calibrated_magnitude'] > 0.0) & (lc_data['calibrated_error'] > 0.0) & (lc_data['fwhm_world'] > 0.0) & (lc_data['fwhm_world'] < 9.0/3600.0)]
filtered_data=filtered_data.copy()
filtered_data['mjd']=Time(filtered_data.date,format='jd').mjd
filtered_data.reset_index(inplace=True,drop=True)
#filtered_data['utc']=pd.to_datetime(filtered_data['date'],unit='D', origin='julian')

#start=Time.now()

# slow method in pandas
# for date in filtered_data.index:
#     med_mask=(abs(filtered_data['date'][date] - filtered_data['date']) < 1)
#     for fil in band:
#         filtered_data.loc[med_mask,fil]=numpy.median(filtered_data[med_mask]['calibrated_magnitude'][(filtered_data[med_mask]['filter']==fil)])

#fast method with numpy arrays
date=numpy.array(filtered_data['date'])
filt=numpy.array(filtered_data['filter'])
mag=numpy.array(filtered_data['calibrated_magnitude'])
dt2=numpy.dtype([('B',numpy.float32),('V',numpy.float32),('R',numpy.float32),('I',numpy.float32),
                ('Be',numpy.float32),('Ve',numpy.float32),('Re',numpy.float32),('Ie',numpy.float32)])
colours=numpy.empty(mag.shape,dtype=dt2)
colours[:]=numpy.nan
for i in range(0,len(filtered_data)):
    for f in band:
        check = numpy.where( (filt == f) &  (numpy.abs(date[i] - date) < med_window) )
        if len(check[0]>0):
            colours[i][f]=numpy.median(mag[check[0]])
            colours[i][f+'e'] = numpy.nanstd(mag[check[0]])/numpy.sqrt(float(len(mag[check[0]])))

        
colour_data=pd.concat((filtered_data,pd.DataFrame(colours)),axis=1)

display(colour_data.sort_values(by='date'))

#end=Time.now()
#print((end-start).sec)

### Lightcurve plot

In [ ]:
def plot_lightcurve(date_range_sel=[min(filtered_data['mjd']),max(filtered_data['mjd'])],error_bars=False):
    global filtered_data
    filtered_data=colour_data[(colour_data['mjd'] > date_range_sel[0]) & (colour_data['mjd'] < date_range_sel[1])]

    color_discrete_map = {'B': 'blue', 'V': 'green', 'R': 'red', 'I':'black'}

    #plot lightcurve
    if error_bars==True:
        fig1=px.scatter(filtered_data,x=filtered_data.mjd,y=filtered_data.calibrated_magnitude,color='filter',color_discrete_map=color_discrete_map,labels={'x':'Date [mjd]'},
                       error_y=filtered_data.calibrated_error)
    else:
        fig1=px.scatter(filtered_data,x=filtered_data.mjd,y=filtered_data.calibrated_magnitude,color='filter',color_discrete_map=color_discrete_map,labels={'x':'Date [mjd]'})
    fig1['layout']['yaxis_title']='Magnitude [mag]'
    fig1['layout']['xaxis_title']=f'Date [mjd]'
    fig1['layout']['yaxis']['autorange'] = "reversed"
    fig1['layout']['xaxis']['tickformat'] = "3f"

    iplot(fig1)
    
interact(plot_lightcurve,date_range_sel=widgets.FloatRangeSlider(value=[min(filtered_data['mjd']),max(filtered_data['mjd'])],min=min(filtered_data['mjd']),max=max(filtered_data['mjd']),step=0.1,
    description='Date range [mjd]:',readout_format='.1f',layout={'width': '600px'},style = {'description_width': 'initial'},continuous_update=False))


### Colour curve plot

In [ ]:
def view_colourcurve(col1='V',col2='I',error_bars=False):
    if error_bars==True:
        fig1=px.scatter(filtered_data,x=filtered_data.mjd,y=filtered_data[col1]-filtered_data[col2],labels={'x':'Date [mjd]', 'y':f'{col1}-{col2} [mag]'},
                    error_y=numpy.sqrt(numpy.square(filtered_data[col1+'e']) + numpy.square(filtered_data[col2+'e'])))
    else:
        fig1=px.scatter(filtered_data,x=filtered_data.mjd,y=filtered_data[col1]-filtered_data[col2],labels={'x':'Date [mjd]', 'y':f'{col1}-{col2} [mag]'})
    fig1['layout']['xaxis']['tickformat'] = "3f"
    fig1['layout']['xaxis_title']=f'Date [mjd]'

    iplot(fig1)

interact(view_colourcurve,col1= widgets.Dropdown(options=(band),value='V',description='Select first colour',style = {'description_width': 'initial'}),
                          col2= widgets.Dropdown(options=(band),value='I',description='Select second colour',style = {'description_width': 'initial'}))


### Colour-magnitude plot

In [ ]:
def plot_cmd(m1='V',col1='V',col2='I',A_v_l=1,A_v_c=numpy.min(filtered_data['V']-filtered_data['I']),A_v_m=numpy.min(filtered_data['V']),error_bars=False):
    jd=Time(filtered_data.date,format='jd') 
    mjd=jd.mjd #plot mjd instead of jd
    if error_bars==True:
        fig1=px.scatter(filtered_data,x=filtered_data[col1]-filtered_data[col2],y=filtered_data[m1],color=mjd,
                   error_y=numpy.sqrt(numpy.square(filtered_data[col1+'e']) + numpy.square(filtered_data[col2+'e'])),error_x=filtered_data[m1+'e'],
                       labels={'x':f'Colour ({col1}-{col2})', 'y':f'Magnitude ({m1}) ','color':'date'})
    else:
        fig1=px.scatter(filtered_data,x=filtered_data[col1]-filtered_data[col2],y=filtered_data[m1],color=mjd,
                        labels={'Y':f'Magnitude ({m1})','x':f'Colour ({col1}-{col2})', 'color':'date'})

    A_3p1 = {'B': 4.7, 'V': 3.55, 'R': 2.66, 'I':1.7}
    
    A_mag_3p1=A_v_l * A_3p1[m1]/A_3p1['V']
    A_col_3p1=A_v_l * ( (A_3p1[col1]/A_3p1['V']) - (A_3p1[col2]/A_3p1['V']) )
        
    xi = A_v_c
    yi = A_v_m
    fig2=px.line(x=(xi,xi+A_col_3p1),y=(yi,yi+A_mag_3p1),labels='R_v=3.1')
    fig2.update_traces(line=dict(color = 'blue',width=3),name='R_v=3.1')

    A_5 = {'B': 3.67, 'V': 3.06, 'R': 2.43, 'I':1.7}
    
    A_mag_5=A_v_l * A_5[m1]/A_5['V']
    A_col_5=A_v_l * ( (A_5[col1]/A_5['V']) - (A_5[col2]/A_5['V']) )
        
    xi = A_v_c
    yi = A_v_m
    fig3=px.line(x=(xi,xi+A_col_5),y=(yi,yi+A_mag_5),labels='R_v=5.0')
    fig3.update_traces(line=dict(color = 'green',width=3),name='R_v=5.0')
   
    fig4 = go.Figure(data=fig2.data  + fig3.data + fig1.data)
    fig4['layout']['yaxis_title']=f'Magnitude ({m1}) [mag]'
    fig4['layout']['xaxis_title']=f'Colour ({col1}-{col2}) [mag]'
    fig4['layout']['yaxis']['autorange'] = "reversed"
    fig4['layout']['coloraxis_colorbar']['title'] = 'Date [mjd]'
    fig4['layout']['coloraxis']['colorscale'] = 'Matter'
    fig4['data'][0]['showlegend']=True
    fig4['data'][1]['showlegend']=True
    fig4.update_layout(legend=dict(yanchor="top",xanchor="right"))
    fig4.update_layout(autosize=False,width=900,height=600)
    fig4.update_layout(legend_title="Extinction Vector")
    
    fig4.show()
    #iplot(fig1) 
    
interact(plot_cmd,m1= widgets.Dropdown(options=(band),value='V',description='Select magnitude',style = {'description_width': 'initial'}),
                         col1= widgets.Dropdown(options=(band),value='V',description='Select first colour',style = {'description_width': 'initial'}),
                          col2= widgets.Dropdown(options=(band),value='I',description='Select second colour',style = {'description_width': 'initial'}),
                        A_v_l=widgets.FloatSlider(value=1,min=0,max=5,step=0.01,description='Av vector start length [mag]:',layout={'width': '600px'},style = {'description_width': 'initial'},continuous_update=False),
                        A_v_c=widgets.FloatSlider(value=numpy.min(filtered_data['V']-filtered_data['I']),min=numpy.min(filtered_data['V']-filtered_data['I'])-1,max=numpy.min(filtered_data['V']-filtered_data['I'])+1,step=0.01,description='Av vector start colour [mag]:',layout={'width': '600px'},style = {'description_width': 'initial'},continuous_update=False),
                        A_v_m=widgets.FloatSlider(value=numpy.min(filtered_data['V']),min=numpy.min(filtered_data['V'])-2,max=numpy.min(filtered_data['V'])+2,step=0.01,description='Av vector start magnitude [mag]:',layout={'width': '600px'},style = {'description_width': 'initial'},continuous_update=False)
        )
   